In [2]:
import sys
import os

sys.path.append(os.path.abspath('..'))

In [3]:
from utils import get_cuda_info

get_cuda_info()

PyTorch version: 2.5.1+cu118
**********
_CUDA version: 
CUDA version:
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Wed_Oct_30_01:18:48_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.6, V12.6.85
Build cuda_12.6.r12.6/compiler.35059454_0

**********
CUDNN version: 90100
Available GPU devices: 1
Device Name: NVIDIA GeForce RTX 4070 Ti SUPER


## Zdobycie danych

In [4]:
import numpy as np

LANDMARK_INDEXES = np.load(os.path.join('..', '..', 'data', 'landmarks', 'combined_selected_points_emotions.npy'))

In [5]:
from utils import load_data, get_selected_landmarks

all_data, all_labels = load_data('real_life_deception_detection')
all_data = get_selected_landmarks(all_data, LANDMARK_INDEXES)

## Preprocessing danych

In [6]:
from utils import preprocess_data

X_train, X_val, X_test, y_train, y_val, y_test = preprocess_data(all_data, all_labels, binarize_labels=False)

In [7]:
print(X_train.shape)
print(y_train.shape)

torch.Size([84, 2125, 154, 2])
torch.Size([84])


In [8]:
from utils import get_class_distribution

get_class_distribution(all_labels)

===> Class distribution <===
0: 60
1: 61
=============><=============


# MODEL TORCH

## Zbudowanie modelu ekstrakcji cech

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [10]:
class LieClassifier(nn.Module):
    def __init__(self, num_landmarks):
        super(LieClassifier, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=2, out_channels=32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        
        # Compute conv_output_size dynamically
        self.conv_output_size = self._get_conv_output_size(num_landmarks)
        
        self.lstm = nn.LSTM(input_size=self.conv_output_size, hidden_size=128, 
                            batch_first=True, bidirectional=True)
        
        self.attn = nn.Linear(256, 1)
        self.fc1 = nn.Linear(256, 64)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(64, 1)

        self._init_weights()

    def _get_conv_output_size(self, num_landmarks):
        # Dummy input: (batch=1, channels=2, landmarks)
        x = torch.zeros(1, 2, num_landmarks)
        x = self.pool1(F.relu(self.conv1(x)))  # -> (1, 32, L1)
        x = self.pool2(F.relu(self.conv2(x)))  # -> (1, 64, L2)
        return x.numel()  # Flattened size: channels * length

    def _init_weights(self):
        for name, param in self.named_parameters():
            if 'weight' in name:
                nn.init.xavier_normal_(param)
            elif 'bias' in name:
                nn.init.constant_(param, 0.1)

    def forward(self, x):
        # x: (batch, frames, landmarks, coordinates)
        batch_size, frames, landmarks, coordinates = x.shape
        
        x = x.view(-1, landmarks, coordinates)  # (batch*frames, landmarks, 2)
        x = x.permute(0, 2, 1)  # (batch*frames, 2, landmarks)
        
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))

        x = x.view(batch_size, frames, -1)  # Flatten: (batch, frames, conv_output_size)

        x, _ = self.lstm(x)
        attn_weights = torch.softmax(self.attn(x), dim=1)
        x = (x * attn_weights).sum(dim=1)

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)  # Raw logits

In [11]:
from torch.optim import Adam

model = LieClassifier(num_landmarks=len(LANDMARK_INDEXES))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pos_weight = torch.tensor([(len(y_train) - y_train.sum()) / y_train.sum()]).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = Adam(model.parameters(), lr=1e-4)

## Trening modelu

In [12]:
from torch.utils.tensorboard import SummaryWriter

RUNS_FOLDER_PATH = os.path.abspath('runs')
writer_path = os.path.join('runs', 'torch_lstm', 'lie_classifier')
writer = SummaryWriter(writer_path)

### Diagnostyka

In [ ]:
from utils.model_functions import overfit_model

writer_diag_path = os.path.join('runs', 'torch_lstm', 'lie_classifier_overfit')
writer_diag = SummaryWriter(writer_diag_path)
model_diag = LieClassifier()
pos_weight_diag = torch.tensor([(len(y_train) - y_train.sum()) / y_train.sum()]).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
criterion_diag = nn.BCEWithLogitsLoss(pos_weight=pos_weight_diag)
optimizer_diag = Adam(model.parameters(), lr=1e-3)

overfit_model(model_diag, criterion_diag, optimizer_diag, X_train, y_train, batch_size=8, writer=writer_diag, epochs=200)

In [13]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [14]:
from utils.model_functions import train_torch_model_binary

train_torch_model_binary(model, criterion, optimizer, X_train, y_train, X_val, y_val, writer=writer, batch_size=32, unbalanced=True, show_prediction_stats=False)


                                          EPOCH STATISTICS                                          
Epoch       : 1
----------------------------------------------------------------------------------------------------
                     TRAINING                                         VALIDATION                    
----------------------------------------------------------------------------------------------------
Loss        : 2.1912                                    Loss        : 0.7139
Accuracy    : 0.4881                                    Accuracy    : 0.5000
Precision   : 0.4875                                    Precision   : 0.2500
Recall      : 0.4881                                    Recall      : 0.5000
F1 Score    : 0.4822                                    F1 Score    : 0.3333
----------------------------------------------------------------------------------------------------
                                          VALIDATION EXTRA                                   

## Ewaluacja modelu

In [15]:
from utils.model_functions import eval_torch_model_binary

eval_torch_model_binary(model, criterion, X_test, y_test)


                                          EPOCH STATISTICS                                          
Epoch       : 1
----------------------------------------------------------------------------------------------------
                                             VALIDATION                                             
----------------------------------------------------------------------------------------------------
Loss        : 0.7067
Accuracy    : 0.5789
Precision   : 0.2895
Recall      : 0.5000
F1 Score    : 0.3667
----------------------------------------------------------------------------------------------------
                                          VALIDATION EXTRA                                          
TP Rate     : 1.0000                                    FP Rate     : 1.0000



# MODEL SEGLEARN

In [9]:
from xgboost import XGBClassifier
from seglearn.pipe import Pype
from seglearn.transform import FeatureRep, Segment

### Przekształcenie danych na wektor płaski połączonych współrzędnych

In [15]:
X_train_np = X_train.numpy()
X_val_np = X_val.numpy()
X_test_np = X_test.numpy()
y_train_np = y_train.numpy()
y_val_np = y_val.numpy()
y_test_np = y_test.numpy()

In [16]:
def flatten_landmarks(data):
    n_samples, n_timesteps, n_landmarks, n_coords = data.shape
    return data.reshape(n_samples, n_timesteps, n_landmarks * n_coords)

X_train_flat = flatten_landmarks(X_train_np)
X_val_flat = flatten_landmarks(X_val_np)
X_test_flat = flatten_landmarks(X_test_np)

In [17]:
print(X_train_flat.shape, y_train_np.shape)

(224, 1679, 956) (224,)


### Budowa modelu

In [18]:
pipe = Pype([
    ("segment", Segment(width=20, step=10)),  # Segmentacja sekwencji
    ("features", FeatureRep()),              # Ekstrakcja cech
    ("xgb", XGBClassifier(
        eval_metric='logloss',
        n_estimators=200
    ))
])

### Trening modelu

In [19]:
pipe.fit(X_train_flat, y_train_np)

Pype(steps=[('segment', Segment(overlap=None, step=10, width=20)),
            ('features',
             FeatureRep(features={'abs_energy': <function abs_energy at 0x0000023A8F32AB60>,
                                  'kurt': <function kurt at 0x0000023A8F32B060>,
                                  'max': <function maximum at 0x0000023A8F32AF20>,
                                  'mean': <function mean at 0x0000023A8F32A7A0>,
                                  'median': <function median at 0x0000023A8F32A840>,
                                  'min': <function minimum at 0x...
                           gamma=None, grow_policy=None, importance_type=None,
                           interaction_constraints=None, learning_rate=None,
                           max_bin=None, max_cat_threshold=None,
                           max_cat_to_onehot=None, max_delta_step=None,
                           max_depth=None, max_leaves=None,
                           min_child_weight=None, missing=nan,
                           monotone_constraints=None, multi_strategy=None,
                           n_estimators=200, n_jobs=None,
                           num_parallel_tree=None, random_state=None, ...))])

### Ewaluacja modelu

In [20]:
val_accuracy = pipe.score(X_val_flat, y_val_np)
test_accuracy = pipe.score(X_test_flat, y_test_np)

print(f"Dokładność na zbiorze walidacyjnym: {val_accuracy:.2f}")
print(f"Dokładność na zbiorze testowym: {test_accuracy:.2f}")

Dokładność na zbiorze walidacyjnym: 0.36
Dokładność na zbiorze testowym: 0.40


# TODYNET

### Przygotowanie danych

In [ ]:
TodyNet_DATA_PATH = os.path.join("..", "..", "src", "external", "TodyNet", "data", "UCR", "SILESIAN_DECEPTION")

os.makedirs(TodyNet_DATA_PATH, exist_ok=True)

In [28]:
X_train_tensor = torch.tensor(X_train_flat, dtype=torch.float32).unsqueeze(1)  # adding channel dimension
X_val_tensor = torch.tensor(X_val_flat, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test_flat, dtype=torch.float32).unsqueeze(1)

# Save the data in PyTorch (.pt) format
torch.save(X_train_tensor, os.path.join(TodyNet_DATA_PATH, 'X_train.pt'))
torch.save(X_val_tensor, os.path.join(TodyNet_DATA_PATH, 'X_valid.pt'))
torch.save(X_test_tensor, os.path.join(TodyNet_DATA_PATH, 'X.pt'))

# Save the labels in PyTorch (.pt) format
torch.save(y_train, os.path.join(TodyNet_DATA_PATH, 'y_train.pt'))
torch.save(y_val, os.path.join(TodyNet_DATA_PATH, 'y_valid.pt'))
torch.save(y_test, os.path.join(TodyNet_DATA_PATH, 'y.pt'))

In [29]:
X_train_tensor.shape

torch.Size([224, 1, 1679, 956])

### Trening modelu [pool_ratio 0.8, ponieważ rozmiar danych jest zbyt duży na 0.2]

In [ ]:
# cd .\src\external\TodyNet\src\ & python train.py --dataset='SILESIAN_DECEPTION' --pool_ratio 0.8